<a href="https://colab.research.google.com/github/AishwaryaHoysal24/PyTorch-Zero-to-GANs/blob/Main/Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Introduction to linear regression and gradient descent
- Implementing a linear regression model using PyTorch tensors
- Training a linear regression model using the gradient descent algorithm
- Implementing gradient descent and linear regression using PyTorch built-in

## Introduction to Linear Regression

In this tutorial, we'll discuss one of the foundational algorithms in machine learning: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall, and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

In [2]:
import numpy as np
import torch

In [3]:
#Training data
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

In [4]:
# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [5]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [6]:
#Linear Reg Model
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.3572, -0.3514,  0.4823],
        [-0.8650, -1.2975, -0.3974]], requires_grad=True)
tensor([ 1.8347, -0.3186], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [7]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

In [8]:
inputs @ w.t() + b

tensor([[ -27.0433, -167.4825],
        [ -30.7235, -218.6444],
        [ -48.3529, -272.4836],
        [ -31.8611, -159.0449],
        [ -22.7835, -212.3776]], grad_fn=<AddBackward0>)

In [9]:
def model(x):
    return x @ w.t() + b #.t method gives the TRANSPOSE of the matrix

In [10]:
#Generate pred
preds=model(inputs)
print(preds)

tensor([[ -27.0433, -167.4825],
        [ -30.7235, -218.6444],
        [ -48.3529, -272.4836],
        [ -31.8611, -159.0449],
        [ -22.7835, -212.3776]], grad_fn=<AddBackward0>)


In [11]:
#Compare Targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to *just work*.

In [12]:
diff=preds-targets
torch.sum(diff * diff) / diff.numel() #MSE

tensor(53733.9922, grad_fn=<DivBackward0>)

In [13]:
#MSE Loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel() #Num of elements in the matrix

In [14]:
#Computing loss
loss=mse(preds, targets)
print(loss) #Lower the loss,better the model

tensor(53733.9922, grad_fn=<DivBackward0>)


In [15]:
#compare gradients
loss.backward()

In [16]:
#Gradients for weight
print(w)
print(w.grad)

tensor([[-0.3572, -0.3514,  0.4823],
        [-0.8650, -1.2975, -0.3974]], requires_grad=True)
tensor([[ -8992.3193, -10442.4209,  -6245.0679],
        [-24912.5156, -27723.4043, -16927.4258]])


In [17]:
print(b)
print(b.grad)

tensor([ 1.8347, -0.3186], requires_grad=True)
tensor([-108.3529, -298.0066])


## Adjust weights and biases to reduce the loss

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's [slope](https://en.wikipedia.org/wiki/Slope) w.r.t. the weights and biases.

If a gradient element is **positive**:

* **increasing** the weight element's value slightly will **increase** the loss
* **decreasing** the weight element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

* **increasing** the weight element's value slightly will **decrease** the loss
* **decreasing** the weight element's value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [18]:
print(w)
#print(w.grad) #inc the weight -->derivative is negative and dec when derivative is +ve
w.grad*1e-5

tensor([[-0.3572, -0.3514,  0.4823],
        [-0.8650, -1.2975, -0.3974]], requires_grad=True)


tensor([[-0.0899, -0.1044, -0.0625],
        [-0.2491, -0.2772, -0.1693]])

In [19]:
with torch.no_grad():
    w -= w.grad * 1e-5 #element wise subtraction
    b -= b.grad * 1e-5

In [20]:
w,b

(tensor([[-0.2672, -0.2470,  0.5447],
         [-0.6159, -1.0202, -0.2281]], requires_grad=True),
 tensor([ 1.8358, -0.3156], requires_grad=True))

In [21]:
#verify the loss again
preds=model(inputs)
loss=mse(preds, targets)
print(loss) #bit less than prev loss

tensor(36391.3828, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [22]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero


## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an _epoch_. Let's train the model for 100 epochs.

In [23]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

Once again, let's verify that the loss is now lower:

In [24]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(164.2258, grad_fn=<DivBackward0>)


The loss is now much lower than its initial value. Let's look at the model's predictions and compare them with the targets.

In [25]:
# Predictions
preds

tensor([[ 60.7853,  74.8149],
        [ 86.2132, 102.0699],
        [103.7673, 122.5304],
        [ 40.4921,  63.3340],
        [ 97.9254, 106.1542]], grad_fn=<AddBackward0>)

In [26]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The predictions are now quite close to the target variables. We can get even better results by training for a few more epochs.

In [27]:
!pip install jovian --upgrade -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 649.1 kB/s eta 0:00:00


In [28]:
import jovian

In [29]:
jovian.commit(project='02-linear-regression')

[jovian] Detected Colab notebook...
[jovian] jovian.commit() is no longer required on Google Colab. If you ran this notebook from Jovian, 
then just save this file in Colab using Ctrl+S/Cmd+S and it will be updated on Jovian. 
Also, you can also delete this cell, it's no longer necessary.
